### Useful import

In [270]:
import pandas as pd
import numpy as np
import glob
import re
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import tree

In [59]:
path_shots = "Data/Team/Regular_Season/shots/*"

df_shots = []


for file in glob.iglob(path_shots):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[31:35]
    df_shots.append(frame)
    
df_shots = pd.concat(df_shots)

df_shots.drop(df_shots.columns[[0]], axis=1, inplace=True)
df_shots.reset_index(inplace=True)
df_shots = df_shots.drop(df_shots[df_shots.Team == 'Durchschnitt'].index)
df_shots.set_index(['Team','Year'], inplace=True)
df_shots.sort_index(inplace=True)
#df_shots.head(10)

In [60]:
path_powerkill = "Data/Team/Regular_Season/powerkill/*"

df_powerkill = []

for file in glob.iglob(path_powerkill):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_powerkill.append(frame)

df_powerkill = pd.concat(df_powerkill)

df_powerkill.drop(df_powerkill.columns[[0]], axis=1, inplace=True)
df_powerkill.reset_index(inplace=True)
df_powerkill = df_powerkill.drop(df_powerkill[df_powerkill.Team == 'Durchschnitt'].index)
df_powerkill.set_index(['Team','Year'], inplace=True)
df_powerkill.sort_index(inplace=True)

#df_powerkill.head(10)

In [61]:
path_goals = "Data/Team/Regular_Season/goals/*"

df_goals = []

for file in glob.iglob(path_goals):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[31:35]
    df_goals.append(frame)

df_goals = pd.concat(df_goals)

df_goals.drop(df_goals.columns[[0]], axis=1, inplace=True)
df_goals.reset_index(inplace=True)
df_goals = df_goals.drop(df_goals[df_goals.Team == 'Durchschnitt'].index)
df_goals.set_index(['Team','Year'], inplace=True)
df_goals.sort_index(inplace=True)

#df_goals.head(10)

In [62]:
path_penalties = "Data/Team/Regular_Season/penalties/*"

df_penalties = []

for file in glob.iglob(path_penalties):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_penalties.append(frame)
    
df_penalties = pd.concat(df_penalties)

df_penalties.drop(df_penalties.columns[[0]], axis=1, inplace=True)
df_penalties.reset_index(inplace=True)
df_penalties = df_penalties.drop(df_penalties[df_penalties.Team == 'Durchschnitt'].index)
df_penalties.set_index(['Team','Year'], inplace=True)
df_penalties.sort_index(inplace=True)

#df_penalties.head(10)

In [63]:
path_powerplay = "Data/Team/Regular_Season/powerplay/*"

df_powerplay = []

for file in glob.iglob(path_powerplay):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_powerplay.append(frame)
    
df_powerplay

df_powerplay = pd.concat(df_powerplay)

df_powerplay.drop(df_powerplay.columns[[0]], axis=1, inplace=True)
df_powerplay.reset_index(inplace=True)
df_powerplay = df_powerplay.drop(df_powerplay[df_powerplay.Team == 'Durchschnitt'].index)
df_powerplay.set_index(['Team','Year'], inplace=True)
df_powerplay.sort_index(inplace=True)

#df_powerplay.head(10)

In [64]:
def timer_to_int(time) :
    m = re.search('(\d+):(\d+)', time)
    return int(m.group(1))*60 + int(m.group(2))

In [65]:
df = pd.concat([df_shots, df_powerkill, df_goals, df_penalties, df_powerplay], axis=1)
df = df.loc[:,~df.columns.duplicated()]
df = df.reset_index().set_index(['Team','Year'])
df['Number of year'] = 1
df.drop(['GP', 'index'],axis=1, inplace=True)
#df.drop(['GP', 'index','PKT', 'PK time/GP','PPT','PP time/GP'],axis=1, inplace=True)
df['SOG'] = df['SOG'].apply(lambda x : int(x[0] + x[2:]))
df['PKT'] = df['PKT'].apply(lambda x : timer_to_int(x))
df['PK time/GP'] = df['PK time/GP'].apply(lambda x : timer_to_int(x))
df['PPT'] = df['PPT'].apply(lambda x : timer_to_int(x))
df['PP time/GP'] = df['PP time/GP'].apply(lambda x : timer_to_int(x))
df
#df.to_csv('Data/Cleaned_Data/team_regular_season.csv')

GF   SOG     S%  SOG/GP  PK SI  PK SI/GP  \
Team                      Year                                              
EHC Biel                  0809  126  1126  11.19      23    306      6.12   
                          0910  134  1340  10.00      27    266      5.32   
                          1011  135  1530   8.82      31    201      4.02   
                          1112  114  1584   7.20      32    200      4.00   
                          1213  160  1733   9.23      35    198      3.96   
                          1314  115  1760   6.53      35    197      3.94   
                          1415  142  1547   9.18      31    167      3.34   
                          1516  128  1741   7.35      35    197      3.94   
EV Zug                    0809  171  1524  11.22      30    289      5.78   
                          0910  165  1881   8.77      38    229      4.58   
                          1011  176  1701  10.35      34    224      4.48   
                          1112  173  1623  10.66      32    221      4.42   
                          1213  172  1515  11.35      30    236      4.72   
                          1314  132  1721   7.67      34    225      4.50   
                          1415  166  1569  10.58      31    211      4.22   
                          1516  161  1667   9.66      33    188      3.76   
Fribourg-Gottéron         0809  154  1550   9.94      31    273      5.46   
                          0910  145  1735   8.36      35    228      4.56   
                          1011  157  1507  10.42      30    208      4.16   
                          1112  156  1586   9.84      32    188      3.76   
                          1213  163  1568  10.40      31    227      4.54   
                          1314  151  1616   9.34      32    196      3.92   
                          1415  144  1630   8.83      33    195      3.90   
                          1516  148  1399  10.58      28    165      3.30   
Genève-Servette HC        0809  157    92  16.67      19    297      5.94   
                          0910  166  1066  15.57      21    250      5.00   
                          1011  129  1250  10.32      25    208      4.16   
                          1112  117  1519   7.70      30    186      3.72   
                          1213  146  1540   9.48      31    249      4.98   
                          1314  158  1558  10.14      31    210      4.20   
...                             ...   ...    ...     ...    ...       ...   
Lausanne HC               1516  123  1553   7.92      31    172      3.44   
SC Bern                   0809  187  1459  12.82      29    262      5.24   
                          0910  154  1558   9.88      31    200      4.00   
                          1011  160  1755   9.12      35    184      3.68   
                          1112  153  1632   9.38      33    195      3.90   
                          1213  169  1792   9.43      36    201      4.02   
                          1314  126  1497   8.42      30    187      3.74   
                          1415  158  1489  10.61      30    184      3.68   
                          1516  152  1702   8.93      34    190      3.80   
SC Rapperswil-Jona Lakers 0809  143  1325  10.79      26    271      5.42   
                          0910  129  1244  10.37      25    196      3.92   
                          1011  138  1199  11.51      24    219      4.38   
                          1112   99  1250   7.92      25    199      3.98   
                          1213  139  1505   9.24      30    206      4.12   
                          1314  117  1559   7.50      31    203      4.06   
                          1415  121  1577   7.67      32    192      3.84   
SCL Tigers                0809  170  1497  11.36      30    304      6.08   
                          0910  141  1277  11.04      26    206      4.12   
                          1011  149  1482  10.05      30    179      3.58   
                          111

In [66]:
df.columns

Index(['GF', 'SOG', 'S%', 'SOG/GP', 'PK SI', 'PK SI/GP', 'PK GA', 'PK GA/GP',
       'PK%', 'PKT', 'PK time/GP', 'SHG', 'PK GF/GP', 'GF/GP', 'GA', 'GA/GP',
       'GF even', 'GF even/GP', 'GA even', 'GA even/GP', 'PPG', 'PPG/GP',
       'PP GA', 'PP GA/GP', 'SOWGF', 'SOWGA', '2' ', '5'', '10'', 'GM', 'MP',
       'PIM Total', 'PP OP', 'PP OP/GP', 'PP%', 'PPT', 'PP time/GP',
       'Number of year'],
      dtype='object')

In [67]:
PO_path = 'Data/Team/Rankings/by_match/playoff/*'
df_match = []

for file in glob.iglob(PO_path):
    f = pd.read_csv(file, ';')
    f['Year'] = file[36:40]
    df_match.append(f)

df_match = pd.concat(df_match)
df_match = df_match[['Home', 'Away', 'Resultat', 'Year']]
df_match['Result'] = [ 'h' if x[1] > x[3] else 'a' for x in df_match["Resultat"]]
#df_match['Series_ID'] = df_match['Year']+'_'+set([df_match['Home'], df_match['Away']])
#df_match['Series_ID'] = [0]*len(df_match)

In [68]:
df_match['Series_ID'] = df_match.apply(lambda x: x.Year + '_' + ' '.join(sorted([x.Home, x.Away])), axis=1)

In [69]:
play_off_series = df_match.groupby('Series_ID').agg({
    'Year': {
        'Year': 'first'
    },
    'Home': {
        'Homefield_advantage': lambda x: x.value_counts().index[0],
        'Opponent 0': 'first'
    },
    'Away': {
        'Opponent 1': 'first'
    },
    'Result': {
        'Winner': lambda x: 0 if x.value_counts().index[0]=='h' else 1
    }
})
play_off_series.columns = play_off_series.columns.droplevel()
#play_off_series

In [70]:
df_match_stats = []

for x in play_off_series.iterrows() :
    year = x[1]['Year']
    opp0 = x[1]['Opponent 0']
    opp1 = x[1]['Opponent 1']
    winner = x[1]['Winner']
    
#    line = df.loc[(opp0,year)]-df.loc[(opp1,year)] if winner == opp0 else df.loc[(opp1,year)]-df.loc[(opp0,year)]
    line = df.loc[(opp0,year)]-df.loc[(opp1,year)]
    aa = pd.DataFrame(line).transpose()
    aa['Series_ID'] = year + '_' + ' '.join(sorted([opp0, opp1]))
    aa.set_index(['Series_ID'],inplace=True)

    df_match_stats = pd.concat([pd.DataFrame(df_match_stats),aa])

cc = pd.concat([play_off_series,df_match_stats],axis=1)
cc['Homefield_advantage'] = cc['Homefield_advantage'] == cc['Opponent 1']
cc.Homefield_advantage = cc.Homefield_advantage.astype(int)
cc

,Opponent 0,Homefield_advantage,Winner,Year,Opponent 1,GF,SOG,S%,SOG/GP,PK SI,...,10',GM,MP,PIM Total,PP OP,PP OP/GP,PP%,PPT,PP time/GP,Number of year
Series_ID,,,,,,,,,,,,,,,,,,,,,
0809_EV Zug Kloten Flyers,Kloten Flyers,0,0,0809,EV Zug,3.0,-76.0,0.80,-1.0,-18.0,...,11.0,-3.0,0.0,3.0,20.0,0.40,0.20,4586.0,92.0,0.0
0809_EV Zug SC Bern,SC Bern,1,0,0809,EV Zug,16.0,-65.0,1.60,-1.0,-27.0,...,2.0,0.0,1.0,-8.0,-39.0,-0.78,5.01,-4220.0,-84.0,0.0
0809_Fribourg-Gottéron HC Davos,HC Davos,0,0,0809,Fribourg-Gottéron,30.0,-178.0,3.47,-4.0,25.0,...,-4.0,1.0,-1.0,-20.0,-24.0,-0.48,3.74,-3443.0,-69.0,0.0
0809_Fribourg-Gottéron ZSC Lions,ZSC Lions,0,0,0809,Fribourg-Gottéron,22.0,121.0,0.59,2.0,-31.0,...,-4.0,0.0,-1.0,-134.0,-9.0,-0.18,4.81,-1737.0,-35.0,0.0
0809_Genève-Servette HC Kloten Flyers,Kloten Flyers,0,0,0809,Genève-Servette HC,17.0,1356.0,-4.65,10.0,-26.0,...,0.0,0.0,-2.0,-120.0,12.0,0.24,3.01,3004.0,60.0,0.0
0809_HC Davos HC Lugano,HC Davos,0,1,0809,HC Lugano,9.0,-236.0,2.53,-5.0,14.0,...,-3.0,-1.0,-1.0,-97.0,-5.0,-0.10,-5.19,36.0,1.0,0.0
0809_HC Davos Kloten Flyers,Kloten Flyers,0,1,0809,HC Davos,-10.0,76.0,-1.39,2.0,-27.0,...,8.0,1.0,0.0,65.0,45.0,0.90,-2.90,7321.0,146.0,0.0
0910_EV Zug Genève-Servette HC,Genève-Servette HC,1,0,0910,EV Zug,1.0,-815.0,6.80,-17.0,21.0,...,3.0,2.0,1.0,126.0,29.0,0.58,-6.62,2381.0,48.0,0.0
0910_EV Zug ZSC Lions,EV Zug,0,1,0910,ZSC Lions,-20.0,288.0,-2.84,6.0,12.0,...,3.0,3.0,0.0,161.0,-6.0,-0.12,2.80,-442.0,-9.0,0.0


In [322]:
y = cc['Winner'].values
X = cc.drop(['Opponent 0', 'Winner', 'Year', 'Opponent 1'], axis=1)
X = X.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
clfn = MLPClassifier(solver='lbfgs', activation='logistic', hidden_layer_sizes=(20,), early_stopping=True)
clfn.fit(X_train,y_train)

clft = tree.DecisionTreeClassifier()
clft.fit(X_train, y_train)

print(roc_auc_score(y_test, clf.predict(X_test)))
print(clfn.predict_proba(X_test))
print(clfn.predict(X_test))
print(y_test)

print(roc_auc_score(y_test, clft.predict(X_test)))
print(clft.predict(X_test))
print(y_test)

0.5
[[ 0.98391991  0.01608009]
 [ 0.84730714  0.15269286]
 [ 0.40985898  0.59014102]
 [ 0.97132171  0.02867829]
 [ 0.57313427  0.42686573]
 [ 0.40985898  0.59014102]]
[0 0 1 0 0 1]
[0 0 1 1 0 0]
1.0
[0 0 1 1 0 0]
[0 0 1 1 0 0]
